# Data Merger -- Construction of the Canton-Year Panel

## Purpose
This notebook constructs the **final canton-year panel dataset**
used throughout the project. It merges multiple raw data sources
(economic, demographic, climatic, and structural variables)
into a single, consistent dataset.

## Key Steps
- Load raw canton-level datasets
- Harmonize identifiers and time coverage
- Handle missing values and inconsistencies
- Construct derived features used in the analysis
- Export the final master panel

## Inputs
- Raw data files in `data/raw/`

## Outputs
- `data/intermediate/master_panel_2015_2024.csv`


In [ ]:
from pathlib import Path

# --------------------------------------------------
# Robust project root detection (local & prof)
# --------------------------------------------------
cwd = Path.cwd().resolve()

# Case 1: notebook launched from project root
if (cwd / "data" / "intermediate").exists():
    ROOT = cwd

# Case 2: notebook launched from notebooks/
elif (cwd.parent / "data" / "intermediate").exists():
    ROOT = cwd.parent

else:
    raise FileNotFoundError(
        f"Cannot locate project root from cwd={cwd}. "
        "Expected 'data/intermediate/' in cwd or parent."
    )

DATA = ROOT / "data"
INTER = DATA / "intermediate"
OUT = DATA / "outputs"

INTER.mkdir(parents=True, exist_ok=True)

print("ROOT :", ROOT)
print("INTER:", INTER)


In [ ]:
# ============================================================
# OUTPUT FOLDERS (FIGURES + TABLES)
# ============================================================
FIG_DIR = OUT / "figures"
TAB_DIR = OUT / "tables"
FIG_DIR.mkdir(parents=True, exist_ok=True)
TAB_DIR.mkdir(parents=True, exist_ok=True)

def save_fig(filename: str, dpi: int = 200):
    """Save current matplotlib figure to Outputs/figures/"""
    path = FIG_DIR / filename
    plt.tight_layout()
    plt.savefig(path, dpi=dpi, bbox_inches="tight")
    print(f"Figure saved: {path}")
    plt.close()

def save_table(df, filename_csv: str, filename_xlsx: str | None = None):
    """Save a DataFrame to Outputs/tables/ as CSV (+ optional XLSX)."""
    csv_path = TAB_DIR / filename_csv
    df.to_csv(csv_path, index=False)
    print(f"Table saved: {csv_path}")
    if filename_xlsx:
        xlsx_path = TAB_DIR / filename_xlsx
        df.to_excel(xlsx_path, index=False)   # requires openpyxl
        print(f"Table saved: {xlsx_path}")


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import re

# All necessary files are loaded here (this is where 'elcom' is defined)

ev = pd.read_csv(INTER / "ev_registrations_per_canton_2015_2024.csv")
motor = pd.read_csv(INTER / "motorization_rate_2015_2024.csv")
gdp = pd.read_csv(INTER / "gdp_per_capita_2015_2024.csv")
pop = pd.read_csv(INTER / "population_balance_cantons_2015_2024.csv")
greens = pd.read_csv(INTER / "cantonal_parliaments_greens_2015_2024.csv")
climate = pd.read_csv(INTER / "canton_climate_co2_panel_2015_2024.csv")
elcom = pd.read_csv(INTER / "elcom_prices_by_operator_year_2014_2025.csv")

# Utility function to clean up canton codes (just in case)
CANTON_MAP = {
    "Zurich":"ZH","Zürich":"ZH","Berne":"BE","Bern":"BE","Lucerne":"LU","Luzern":"LU",
    "Uri":"UR","Schwyz":"SZ","Obwald":"OW","Obwalden":"OW","Nidwald":"NW","Nidwalden":"NW",
    "Glaris":"GL","Glarus":"GL","Zoug":"ZG","Zug":"ZG","Fribourg":"FR","Freiburg":"FR",
    "Soleure":"SO","Solothurn":"SO","Bâle-Ville":"BS","Basel-Stadt":"BS",
    "Bâle-Campagne":"BL","Basel-Landschaft":"BL","Schaffhouse":"SH","Schaffhausen":"SH",
    "Appenzell Rh.-Ext.":"AR","Appenzell Ausserrhoden":"AR","Appenzell Rh.-Int.":"AI",
    "Appenzell Innerrhoden":"AI","Saint-Gall":"SG","St. Gallen":"SG","Grisons":"GR",
    "Graubünden":"GR","Argovie":"AG","Aargau":"AG","Thurgovie":"TG","Thurgau":"TG",
    "Tessin":"TI","Ticino":"TI","Vaud":"VD","Valais":"VS","Wallis":"VS",
    "Neuchâtel":"NE","Genève":"GE","Geneva":"GE","Jura":"JU"
}

def map_canton_code(x):
    if pd.isna(x): return None
    x = str(x).strip()
    if x in CANTON_MAP: return CANTON_MAP[x]
    if re.fullmatch(r"[A-Z]{2}", x): return x
    return None

# Ensure that there is only clean canton codes
for d in [ev, gdp, pop, greens, climate]:
    if "canton" in d.columns and "canton_code" not in d.columns:
        d["canton_code"] = d["canton"].apply(map_canton_code)

#  Mapping Operators -> Cantons (ELCOM)

# Dictionary for major operators
operator_to_canton = {
    "BKW": "BE", "Bernische Kraftwerke": "BE",
    "EKZ": "ZH", "Elektrizitätswerke des Kantons Zürich": "ZH", "Stadt Zürich": "ZH",
    "SIG": "GE", "Services Industriels de Genève": "GE",
    "Groupe E": "FR", 
    "Romande Energie": "VD",
    "CKW": "LU",
    "AXPO": "AG", "AEW": "AG",
    "EWB": "BE", "IWB": "BS", "AIL": "TI", "FMV": "VS",
    "EGL": "GL", "Repower": "GR", "SN Energie": "SG",
    "EKS": "SH", "EWA": "UR", "WWZ": "ZG"
}

def find_canton_from_operator(op_name):
    op_name = str(op_name)
    for key, val in operator_to_canton.items():
        if key.lower() in op_name.lower():
            return val
    return None

# Apply the mapping to the 'elcom' DataFrame, which is now loaded.
elcom["canton_code_infer"] = elcom["operatorLabel"].apply(find_canton_from_operator)

# A. Price per canton (average of all operators in the district)
elcom_canton = (
    elcom.groupby(["year", "canton_code_infer"])["avg_price_after_discount_cts_per_kwh"]
    .mean()
    .reset_index()
    .rename(columns={
        "avg_price_after_discount_cts_per_kwh": "price_elcom_cts",
        "canton_code_infer": "canton_code"
    })
)

# B. National Price (Fallback in case some prices are missing)
elcom_national = (
    elcom.groupby("year")["avg_price_after_discount_cts_per_kwh"]
    .mean()
    .reset_index()
    .rename(columns={"avg_price_after_discount_cts_per_kwh": "price_national"})
)

# MERGE GLOBAL

df = ev.copy()

# Merging economic and demographic data
df = df.merge(motor[["canton_code","year","motorization_rate_per_1000"]],
              on=["canton_code","year"], how="left")

df = df.merge(gdp[["canton_code","year","gdp_per_capita_chf"]],
              on=["canton_code","year"], how="left")

df = df.merge(pop[["canton_code","year","pop_dec31","arrivals","departures"]],
              on=["canton_code","year"], how="left")

df = df.merge(greens[["canton_code","year","greens_share"]],
              on=["canton_code","year"], how="left")

df = df.merge(climate[["canton_code","year","summer_temp_c","winter_temp_c","co2_emissions_mt"]],
              on=["canton_code","year"], how="left")

# Electricity price merger
# 1. We try to stick to the specific price for the canton.
df = df.merge(elcom_canton, on=["year", "canton_code"], how="left")

# 2. We stick the national price on everything.
df = df.merge(elcom_national, on="year", how="left")

# 3. We fill in the gaps in the regional price with the national price.
df["price_elcom_cts"] = df["price_elcom_cts"].fillna(df["price_national"])
df = df.drop(columns=["price_national"])

# EXPORT FINAL

out = INTER / "master_panel_2015_2024.csv"
df.to_csv(out, index=False)

print("MASTER DATASET CREATED:", out)
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())
print(df.head())

# STEP 1: Verifying the quality of the master dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

master_path = INTER / "master_panel_2015_2024.csv"
print("Loading :", master_path.resolve())

assert master_path.exists(), f"Can't find: {master_path}"

df = pd.read_csv(master_path)

print("\n=== General Overview ===")
print(df.head())

print("\n=== Columns ===")
print(df.columns.tolist())

print("\n=== Shape ===")
print(df.shape)

print("\n=== Years ===")
print("min year:", df["year"].min(), " | max year:", df["year"].max())

print("\n=== Number of cantons ===")
print(df["canton_code"].nunique(), "/", 26)

print("\n=== Number of lines par canton ===")
print(df.groupby("canton_code").size())


# ----------------------------------------------------------
# Checking that there are NO duplicates for canton+year
# ----------------------------------------------------------
print("\n=== Checking for duplicates by canton/year ===")
dup = df[df.duplicated(subset=["canton_code","year"], keep=False)]
if len(dup) == 0:
    print("✔ No duplicates -> perfect.")
else:
    print("Duplicates detected :")
    display(dup)

# ----------------------------------------------------------
# Analysis of missing values
# ----------------------------------------------------------
print("\n=== % of missing values per column ===")
na = df.isna().mean().sort_values(ascending=False)
print(na)

print("\nColumns with missing values :")
print(na[na > 0])

# ----------------------------------------------------------
# Descriptive statistics
# ----------------------------------------------------------
print("\n=== Descriptive statistics ===")
display(df.describe(include="all"))
desc = df.describe(include="all").reset_index()
save_table(desc, "Data Quality Description.csv", "Data Quality Description.xlsx")


# ----------------------------------------------------------
# Quick visualisation through histograms of our important features
# ----------------------------------------------------------
cols_to_plot = [
    "ev_reg_share",
    "motorization_rate_per_1000",
    "gdp_per_capita_chf",
    "price_elcom_cts",
    "summer_temp_c",
    "winter_temp_c",
    "co2_emissions_mt",
    "greens_share"
]

print("\n=== Histograms of important features ===")

for col in cols_to_plot:
    if col not in df.columns:
        print(f"Missing columns : {col}")
        continue

    plt.figure(figsize=(6,4))
    plt.hist(df[col].dropna(), bins=20, edgecolor="black")
    plt.title(f"Histogram — {col}")
    plt.xlabel(col)
    plt.ylabel("Frequency")
    plt.grid(True, alpha=0.3)
    save_fig("Histogram of important features.png")

# ----------------------------------------------------------
# "EV share per year" visualization
# ----------------------------------------------------------
plt.figure(figsize=(8,5))
df.groupby("year")["ev_reg_share"].mean().plot(marker='o')
plt.title("Average EV Adoption per year")
plt.xlabel("Year")
plt.ylabel("EV share (%)")
plt.grid(True)
save_fig("EV Share Histogram.png")

# ----------------------------------------------------------
# EV share per canton (boxplot)
# ----------------------------------------------------------
plt.figure(figsize=(14,5))
df.boxplot(column="ev_reg_share", by="canton_code", rot=90)
plt.title("Distribution of EV adoption by canton")
plt.suptitle("")
plt.xlabel("Canton")
plt.ylabel("EV share (%)")
plt.grid(True)
save_fig("EV Share Boxplot by Canton.png")

print("\n=== Our dataset in now ready to be modelized ===")
